In [19]:
import sys
import os
import pandas as pd
import itertools
import argparse
from Bio.Seq import Seq
from Bio import SeqIO
from alternating_string import is_alternating

def filter_tails(sequence, tail, chrom, start, end, strand, records, N = False) :

    """
    A function to filter reads where the length of the tail is not equal to the edit distance to the reference seq.

    Also I will filter sequences based on 4 preceeding nucleotides before tail to remove possible sequencing errors. 
    (must be the same NT repeated N times)

    Exceptions : 
    1. If the tail length is <= 2 keep it 
    2. If the tail is homogenously one nucleotide aka TTTTTTT or AAAAAAAA keep it 
    """

    tail_length = len(tail)

    if N : 
        N = N
    else : 
        N = 4

    print(f"Sequence: {sequence}")
    print(f"Tail: {tail}")

    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

    seq_minus_tail = sequence[:-int(tail_length)]
    
    lastN = seq_minus_tail[-int(N):]
    #print(f"Last {N} nucleotides: {lastN}")

    if len(set(lastN)) == 1 :
        print(f"Last 4 nucleodies are the same ({lastN})....fail")
        return False

    # len(set(tail)) == 1 or 
    if tail_length == 1 or is_alternating(tail) : 
        return True
    
    else :
        if strand == "+" :
            ref = str(records[chrom][start : end + tail_length].seq)
        else : 
            fwd = records[chrom][start - tail_length : end ].seq
            seq_obj = Seq(fwd)
            ref = str(seq_obj.reverse_complement())
        
        edit_distance = sum(1 for a, b in zip(sequence, ref) if a != b)

        if len(tail) >= 4 and len(set(tail)) == 1 : 
            return True 

        if edit_distance == tail_length :
            print(f"Edit distance == tail length {sequence} (seq) vs. {ref} (ref) with tail {tail}..pass")
            return True
            
        else : 
            print(f"Edit distance does not equal tail length: {sequence} (seq) vs. {ref} (ref) with tail {tail}...return false")
            return False

    #print("Other tail not meeting some specification...fail")
    return False


In [20]:
file = '/fs/ess/PCON0160/ben/projects/2024_A_disl2_degrades_piRNAs/results_time_course/20240428/tailor_transcripts/files/TOFU5_Degron_rep1_TP6_S21_R1.trimmed.uniq.xartifacts.v0.m1000.transcripts.bed.tsv'
records = SeqIO.to_dict(SeqIO.parse('/fs/ess/PCON0160/ben/genomes/c_elegans/WS279/piRNA.15nt_3p_extension.fa', 'fasta'))

In [21]:
with open(file, 'r') as f : 
    for line in f : 
        if not line.startswith("gene") :
            info = line.strip().split("\t") 
            chrom = info[0]
            start = int(info[1])
            end = int(info[2])
            seq = info[3]
            count = info[4]
            strand = info[5]
            tail = info[7]
            
            if not tail == "*" :
                if seq == "TGTTTTGTTCAGAAATGCGGG" : 
                    filter_tails(seq, tail, chrom, start, end, strand, records, N = False)
            
        

Sequence: TGTTTTGTTCAGAAATGCGGG
Tail: GG
Edit distance does not equal tail length: TGTTTTGTTCAGAAATGCGGG (seq) vs. TGTTTTGTTCAGAAATGCGTG (ref) with tail GG...return false
